In [4]:
!pip install pandas
!pip install tables

     |████████████████████████████████| 4.3MB 3.9MB/s eta 0:00:01


In [1]:
import os
import pandas as pd

In [5]:
metadata_path = "./Pipe/metadata_combined.h5"
ngram_counts = "./ngrams_count.csv"
citation_counts = "./Pipe/citation_count.csv"

# Load metadata
metadata_df = pd.read_hdf(metadata_path)
metadata_df.reset_index(inplace=True)

# Load dataframes withh counts
ngrams_df = pd.read_csv(ngram_counts)
citations_df = pd.read_csv(citation_counts)
ngrams_df.head()

,article_id,Culture,Culture_core,Relational,Relational_core,Demographic,Demographic_core
0,journal-article-10.2307_2065002,9,3,11,5,10,7
1,journal-article-10.2307_3380821,10,6,27,7,7,5
2,journal-article-10.2307_2095822,57,6,61,24,26,61
3,journal-article-10.2307_2631839,2,0,4,0,0,0
4,journal-article-10.2307_40836133,74,52,91,28,30,7


In [6]:
counts_df = ngrams_df.merge(citations_df, on="article_id")
counts_df.head()

,article_id,Culture,Culture_core,Relational,Relational_core,Demographic,Demographic_core,word_count,cultural_author_count,demographic_author_count,relational_author_count
0,journal-article-10.2307_2065002,9,3,11,5,10,7,3529,0,0,0
1,journal-article-10.2307_3380821,10,6,27,7,7,5,5195,0,0,0
2,journal-article-10.2307_2095822,57,6,61,24,26,61,7100,0,0,0
3,journal-article-10.2307_2631839,2,0,4,0,0,0,315,0,0,0
4,journal-article-10.2307_40836133,74,52,91,28,30,7,7110,0,0,0


In [7]:
perspectives = ["Culture", "Relational", "Demographic"]
d = {}
for p in perspectives:
    d[p] = "{}_ngram_count".format(p.lower())
    
for p in perspectives:
    d[p + "_core"] = "{}_ngram_count".format(p.lower())

In [8]:
counts_df = counts_df.rename(columns=d)
counts_df.head()

,article_id,culture_ngram_count,culture_ngram_count,relational_ngram_count,relational_ngram_count,demographic_ngram_count,demographic_ngram_count,word_count,cultural_author_count,demographic_author_count,relational_author_count
0,journal-article-10.2307_2065002,9,3,11,5,10,7,3529,0,0,0
1,journal-article-10.2307_3380821,10,6,27,7,7,5,5195,0,0,0
2,journal-article-10.2307_2095822,57,6,61,24,26,61,7100,0,0,0
3,journal-article-10.2307_2631839,2,0,4,0,0,0,315,0,0,0
4,journal-article-10.2307_40836133,74,52,91,28,30,7,7110,0,0,0


In [9]:
merged_df = counts_df.merge(metadata_df[["file_name", "primary_subject", 'year', 'journal_title', 'journal_id']], how="left", left_on="article_id", right_on="file_name")
merged_df = merged_df.drop("file_name", axis=1)
merged_df.head()

,article_id,culture_ngram_count,culture_ngram_count,relational_ngram_count,relational_ngram_count,demographic_ngram_count,demographic_ngram_count,word_count,cultural_author_count,demographic_author_count,relational_author_count,primary_subject,year,journal_title,journal_id
0,journal-article-10.2307_2065002,9,3,11,5,10,7,3529,0,0,0,Sociology,1978,Contemporary Sociology,j100496
1,journal-article-10.2307_3380821,10,6,27,7,7,5,5195,0,0,0,Management & Organizational Behavior,1986,Public Productivity & Management Review,j100852
2,journal-article-10.2307_2095822,57,6,61,24,26,61,7100,0,0,0,Sociology,1983,American Sociological Review,j100080
3,journal-article-10.2307_2631839,2,0,4,0,0,0,315,0,0,0,Management & Organizational Behavior,1985,Management Science,j100269
4,journal-article-10.2307_40836133,74,52,91,28,30,7,7110,0,0,0,Management & Organizational Behavior,2005,MIR: Management International Review,j50000481


In [11]:
merged_df.to_csv("counts_and_subject.csv", index=False)

In [20]:
merged_df[["cultural_author_count", "demographic_author_count", "relational_author_count", "journal_title", "year"]].groupby(["journal_title", "year"]).sum()

cultural_author_count  \
journal_title                              year                          
Academy of Management Learning & Education 2002                      6   
                                           2003                      3   
                                           2004                      2   
                                           2005                      1   
                                           2006                      3   
                                           2007                      1   
                                           2008                      0   
                                           2009                      1   
                                           2010                      4   
                                           2011                      8   
                                           2012                      7   
Academy of Management Perspectives         1983                      0   
                                           1990                      0   
                                           1991                      0   
                                           1993                      0   
                                           1996                      0   
                                           1998                      0   
                                           1999                      0   
                                           2000                      0   
                                           2001                      0   
                                           2002                      0   
                                           2003                      0   
                                           2004                      0   
                                           2005                      0   
                                           2006                      0   
                                           2007                      1   
                                           2008                      2   
                                           2009                     10   
                                           2010                      3   
                                           2011                      0   
...                                                                ...   
Theory and Society                         2008                     42   
                                           2009                      8   
                                           2010                      3   
                                           2011                     17   
                                           2012                      0   
                                           2013                      3   
                                           2014                      0   
Tourism and Hospitality Research           1999                      0   
                                           2000                      0   
                                           2001                      0   
                                           2002                      0   
                                           2003                      0   
                                           2004                      0   
                                           2005                      0   
                                           2006                      0   
                                           2007                      0   
                                           2008                      1   
                                           2009                      1   
                                           2010                      0   
                                           2011                      0   
                                           2012                      0   
                                           2013      